<h1 align=center><font size = 5>Blah, blah, blah</font></h1>

## Introduction

Blah, blah, blah, blah

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

1. [Data Cleaning](#0)<br>
2. [Data Preprocessing](#1)<br>
3. [Visualizing Data](#2)<br>
3. [Modeling](#3)<br>
4. [Blah, blah, blah](#4) <br>
</div>
<hr>

In [1]:
spark

In [2]:
#import statements
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import *
from datetime import datetime
from pyspark.sql.types import IntegerType, StructType, StructField, StringType
import time

In [3]:
spark = SparkSession.builder.appName('Final_project').getOrCreate()
#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '6'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])

sc = spark.sparkContext

In [4]:
spark.sparkContext.getConf().getAll()

[('spark.driver.port', '35340'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.yarn.jars',
  'local:/opt/cloudera/parcels/CDH-6.3.0-1.cdh6.3.0.p0.1279813/lib/spark/jars/*,local:/opt/cloudera/parcels/CDH-6.3.0-1.cdh6.3.0.p0.1279813/lib/spark/hive/*'),
 ('spark.ui.proxyBase', '/proxy/application_1577383759214_3746'),
 ('spark.yarn.appMasterEnv.MKL_NUM_THREADS', '1'),
 ('spark.sql.queryExecutionListeners',
  'com.cloudera.spark.lineage.NavigatorQueryListener'),
 ('spark.executor.memory', '5g'),
 ('spark.lineage.log.dir', '/var/log/spark/lineage'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'md01.rcc.local,md02.rcc.local'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.executorEnv.PYTHONPATH',
  '/opt/cloudera/parcels/CDH/lib/spark/python/lib/py4j-0.10.7-src.zip:/opt/cloudera/parcels/CDH/lib/spark/python/lib/pyspark.zip<CPS>/opt/cloudera/parcels/CDH-6.3.0-1.cdh6.3.0.p0.1279813/lib/spark/python/lib/py4j-0.10.7-sr

In [5]:
#Datapath
path_comments = "/user/rroongseang/bigdata/parquet/comments_parquet/"
path_users = "/user/rroongseang/bigdata/users/"

In [6]:
#Read data
df = sqlContext.read.parquet(path_comments)
users = spark.read.csv(path_users+"RA.2019-09.csv", inferSchema=True, header=True)
botusers = sqlContext.read.csv(path_users+'bot_userdata.csv',header=True)

# Data Cleaning <a id="0"></a>

In [7]:
df.count()

86633134

In [8]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+--------+------+----------------------+-----------------+----+----------------+-----------+-------------+--------+------+---+-------+--------+---------+------------+-----+------------+---------+------------+--------+
|archived|author|author_flair_css_class|author_flair_text|body|controversiality|created_utc|distinguished|   downs|gilded| id|link_id|    name|parent_id|retrieved_on|score|score_hidden|subreddit|subreddit_id|     ups|
+--------+------+----------------------+-----------------+----+----------------+-----------+-------------+--------+------+---+-------+--------+---------+------------+-----+------------+---------+------------+--------+
|66401016|     0|              75695490|         75551029|   0|               0|          0|     84850803|84468358|     0|  0|      0|84468358|        0|       18219|    0|    76923215|        0|           0|69620202|
+--------+------+----------------------+-----------------+----+----------------+-----------+-------------+--------+------+---+--

In [9]:
#dropping several columns with missing and irrelevant data
#subreddit/subreddit_id is the same for all comments since we are only looking in r/politics.
#author flairs are texts + images next to a username that shows up when they post a comment. Most users don't use flairs
#name is a unique identifier that is mostly null.
#id is a unique identifier for the commment and does not add any value to analysis
df = df.drop('archived','author_flair_css_class','author_flair_text',
             'subreddit','subreddit_id','name','score_hidden','id')

In [10]:
#Convert column types to integers and timestamps

df = df.withColumn("ups", df["ups"].cast(IntegerType())) 
df = df.withColumn("downs", df["downs"].cast(IntegerType()))
df = df.withColumn("gilded", df["gilded"].cast(IntegerType()))
df = df.withColumn("score", df["score"].cast(IntegerType()))
df = df.withColumn("retrieved_on", df["retrieved_on"].cast(IntegerType()))
df = df.withColumn("controversiality", df["controversiality"].cast(IntegerType()))
df = df.withColumn('created_utc',df["created_utc"].cast(IntegerType()))
df = df.withColumn('retrieved_on',df["retrieved_on"].cast(IntegerType()))


df = df.withColumn('created_utc',to_timestamp(df["created_utc"]))
df = df.withColumn('retrieved_on',to_timestamp(df["retrieved_on"]))

In [11]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: integer (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: integer (nullable = true)
 |-- gilded: integer (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: integer (nullable = true)
 |-- ups: integer (nullable = true)



In [12]:
#fill upvote and downvote null values with zeros
df = df.fillna({ 'ups':0, 'downs':0 })

In [13]:
#Rename a user and botuser columns to match for union of dataframe
users = users.selectExpr('id',"name as username", "created_utc as acct_creation",
                         "updated_on","comment_karma","link_karma")

botusers = botusers.selectExpr('username',"post_karma as link_karma", "comment_karma",
                                 "cake_day as acct_creation","is_bot")

In [14]:
import pyspark.sql.functions as f
#Convert account creation and updated on to timestampps
users = users.withColumn('acct_creation',to_timestamp(users["acct_creation"]))
users = users.withColumn('updated_on',to_timestamp(users["updated_on"]))

#Add column is_bot to users dataframe
users = users.withColumn('is_bot',f.lit('False'))

In [15]:
#Cast link karma and comment karma as integer type
#Convert account creation and updated on to timestamps
botusers = botusers.withColumn('link_karma',botusers["link_karma"].cast(IntegerType()))
botusers = botusers.withColumn('comment_karma',botusers["comment_karma"].cast(IntegerType()))
botusers = botusers.withColumn('acct_creation',botusers["acct_creation"].cast(IntegerType()))
botusers = botusers.withColumn('acct_creation',to_timestamp(botusers["acct_creation"]))

In [16]:
#remove rows from user df that exists in botusers
users = users.join(botusers, users.username==botusers.username, "leftanti")

In [17]:
#Append users dataframe with botusers dataframe 
all_users = users.select('username','acct_creation','comment_karma','link_karma','is_bot')\
            .union(botusers.select('username','acct_creation','comment_karma','link_karma','is_bot'))

In [18]:
all_users.printSchema()

root
 |-- username: string (nullable = true)
 |-- acct_creation: timestamp (nullable = true)
 |-- comment_karma: integer (nullable = true)
 |-- link_karma: integer (nullable = true)
 |-- is_bot: string (nullable = true)



In [19]:
#Checking to see if one of the Russian banned accts is in the users list
all_users.filter(all_users.username == 'MiraranaMogra').show()

+-------------+-------------------+-------------+----------+------+
|     username|      acct_creation|comment_karma|link_karma|is_bot|
+-------------+-------------------+-------------+----------+------+
|MiraranaMogra|2015-05-14 09:58:34|          103|      4902|  True|
+-------------+-------------------+-------------+----------+------+



In [24]:
#Combining data frames
df_raw = df.join(all_users, df.author == users.username, 'left')

In [26]:
#Number of comments matched with users dataframe
df_raw.filter(users.username.isNotNull()).count()

69638017

In [27]:
#drop rows where comments authors do not appear in users dataframe
df_raw = df_raw.filter(df_raw.is_bot.isNotNull())
df_raw.count()

69638017

In [28]:
#number of banned account comments in dataframe
df_raw.filter(df_raw.is_bot == 'True').count()

110

# Data Preprocessing <a id="1"></a>

## Pipeline

| <p align="left">**Step**</p>| <p align="left">**Action**</p>|<p align="left">**Example**</p>|
|---|---|---|
| <p align="left">Step 0</p> | <p align="left">Raw Comment</p>  | <p align="left">"Almost 1/3 of registered voters in CA are republican, though..."</p>  |
| <p align="left">Step 1</p> | <p align="left">Put text into lower case</p>  | <p align="left">"almost 1/3 of registered voters in ca are republican, though..."</p>  |
| <p align="left">Step 2</p>  | <p align="left">Remove Non-Text Characters</p>  | <p align="left">"almost of registered voters in ca are republican though"</p> |
| <p align="left">Step 3</p>  | <p align="left">Tokenize Text</p>  | <p align="left">["almost", "of", "registered", "voters", "in", "ca", "are", "republican", "though"]</p> |
| <p align="left">Step 4</p>  | <p align="left">Remove Stopwords</p>  | <p align="left">["almost", "registered", "voters", "ca", "republican"]</p> |
| <p align="left">Step 5</p>  | <p align="left">Stem using Porter Stemmer</p>  | <p align="left">["almost", "regist", "voter", "ca", "republican"]</p>

In [29]:
#Extracting comment posted datetime to new column format HHMMSS
df_raw = df_raw.withColumn('created_time',concat(format_string("%02d",hour(df_raw['created_utc'])),
                                                     format_string("%02d",minute(df_raw['created_utc'])),
                                                     format_string("%02d",second(df_raw['created_utc']))))
df_raw = df_raw.withColumn('created_time',df_raw['created_time'].cast(IntegerType()))

In [32]:
df_raw.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: integer (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: integer (nullable = false)
 |-- gilded: integer (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: integer (nullable = true)
 |-- ups: integer (nullable = false)
 |-- username: string (nullable = true)
 |-- acct_creation: timestamp (nullable = true)
 |-- comment_karma: integer (nullable = true)
 |-- link_karma: integer (nullable = true)
 |-- is_bot: string (nullable = true)
 |-- created_time: integer (nullable = true)



In [34]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def getInterval(time):
    start = int(time)
    return str(start)+"-"+str(start+1)

getIntervalUdf = udf(getInterval,StringType())

df_raw = df_raw.withColumn('year', year(col('created_utc')))
df_raw = df_raw.withColumn('month', month(col('created_utc')))
df_raw = df_raw.withColumn('time', hour(col('created_utc')))

df_raw = df_raw.withColumn("interval",getIntervalUdf("time"))

In [35]:
#remove formatting characters
df_raw = df_raw.withColumn('body_vec', f.regexp_replace('body', "[^a-zA-Z0-9\\s]", ""))

#Next, we will put all letters into lower-case
df_raw = df_raw.withColumn('body_vec', lower(col('body_vec')))

In [36]:
df_raw.show(5)

+------------+--------------------+----------------+-------------------+-------------+-----+------+---------+----------+-------------------+-----+---+------------+-------------------+-------------+----------+------+------------+----+-----+----+--------+--------------------+
|      author|                body|controversiality|        created_utc|distinguished|downs|gilded|  link_id| parent_id|       retrieved_on|score|ups|    username|      acct_creation|comment_karma|link_karma|is_bot|created_time|year|month|time|interval|            body_vec|
+------------+--------------------+----------------+-------------------+-------------+-----+------+---------+----------+-------------------+-----+---+------------+-------------------+-------------+----------+------+------------+----+-----+----+--------+--------------------+
|---JustMe---|Yes. $16 million ...|               0|2016-05-09 17:52:25|         null|    0|     0|t3_4ikml4|t1_d2z41q9|2016-06-12 10:36:52|    4|  4|---JustMe---|2014-02-17 2

In [37]:
from pyspark import keyword_only
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
#import nltk
from nltk.stem.porter import PorterStemmer

#this code courtesy of http://michael-harmon.com/blog/SentimentAnalysisP2.html

class PorterStemming(Transformer, HasInputCol, HasOutputCol):
    """
    PosterStemming class using the NLTK Porter Stemmer
    
    This comes from https://stackoverflow.com/questions/32331848/create-a-custom-transformer-in-pyspark-ml
    Adapted to work with the Porter Stemmer from NLTK.
    """
    
    @keyword_only
    def __init__(self, 
                 inputCol  : str = None, 
                 outputCol : str = None, 
                 min_size  : int = None):
        """
        Constructor takes in the input column name, output column name,
        plus the minimum legnth of a token (min_size)
        """
        # call Transformer classes constructor since were extending it.
        super(Transformer, self).__init__()

        # set Parameter objects minimum token size
        self.min_size = Param(self, "min_size", "")
        self._setDefault(min_size=0)

        # set the input keywork arguments
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

        # initialize Stemmer object
        self.stemmer  = PorterStemmer()

        
    @keyword_only
    def setParams(self, 
                  inputCol  : str = None, 
                  outputCol : str = None, 
                  min_size  : int = None
      ) -> None:
        """
        Function to set the keyword arguemnts
        """
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    

    def _stem_func(self, words  : list) -> list:
        """
        Stemmer function call that performs stemming on a
        list of tokens in words and returns a list of tokens
        that have meet the minimum length requiremnt.
        """
        # We need a way to get min_size and cannot access it 
        # with self.min_size
        min_size       = self.getMinSize()

        # stem that actual tokens by applying 
        # self.stemmer.stem function to each token in 
        # the words list
        stemmed_words  = map(self.stemmer.stem, words)

        # now create the new list of tokens from
        # stemmed_words by filtering out those
        # that are not of legnth > min_size
        filtered_words = filter(lambda x: len(x) > min_size, stemmed_words)

        return list(filtered_words)
    
    def _transform(self, df: DataFrame) -> DataFrame:
        """
        Transform function is the method that is called in the 
        MLPipleline.  We have to override this function for our own use
        and have it call the _stem_func.

        Notice how it takes in a type DataFrame and returns type Dataframe
        """
        # Get the names of the input and output columns to use
        out_col       = self.getOutputCol()
        in_col        = self.getInputCol()

        # create the stemming function UDF by wrapping the stemmer 
        # method function
        stem_func_udf = F.udf(self._stem_func, ArrayType(StringType()))
        
        # now apply that UDF to the column in the dataframe to return
        # a new column that has the same list of words after being stemmed
        df2           = df.withColumn(out_col, stem_func_udf(df[in_col]))

        return df2
  
  
    def setMinSize(self,value):
        """
        This method sets the minimum size value
        for the _paramMap dictionary.
        """
        self._paramMap[self.min_size] = value
        return self

    def getMinSize(self) -> int:
        """
        This method uses the parent classes (Transformer)
        .getOrDefault method to get the minimum
        size of a token.
        """
        return self.getOrDefault(self.min_size)


In [38]:
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover, IDF, StringIndexer
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="body_vec", outputCol="body_vec_token")
remover = StopWordsRemover(inputCol = "body_vec_token", outputCol = "body_vec_token_nosw")
stemmer = PorterStemming(inputCol = "body_vec_token_nosw", outputCol = "body_vec_cleaned")
hashingTF = HashingTF(inputCol="body_vec_cleaned", outputCol="body_vec_tf", numFeatures=100)
idf = IDF(inputCol="body_vec_tf", outputCol="body_vec_tfidf")
label_stringIdx = StringIndexer(inputCol = "is_bot", outputCol = "label")

pipeline = Pipeline(stages=[tokenizer, remover, stemmer, hashingTF,idf, label_stringIdx])

In [ ]:
%time nlpdf = pipeline.fit(df_raw).transform(df_raw)

In [31]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['controversiality', 'ups', 'downs', 'gilded', 
                                       'interval', 'comment_karma','link_karma',
                                        'score'], outputCol='features')
nlpdf = assembler.transform(nlpdf)

In [32]:
# nlpdf = df_raw.drop('body_vec')
# nlpdf = df_raw.drop('body_vec_token')
# nlpdf = df_raw.drop('body_vec_token_nosw')
# nlpdf = df_raw.drop('body_vec_cleaned')

In [33]:
nlpdf.select('body_vec_tfidf').show(5)

+--------------------+
|      body_vec_tfidf|
+--------------------+
|(100,[1,10,30],[2...|
|(100,[30,39,40,44...|
|(100,[6,48,49,70,...|
|(100,[19,30,38,43...|
|(100,[3,4,8,9,10,...|
+--------------------+
only showing top 5 rows



In [ ]:
nlpdf.printSchema()

In [ ]:
#split data into train and test and ensure label classes are stratified
#train_df_notbanned, test_df_notbanned = nlpdf.filter(nlpdf.label==0).sample(False,0.0001,seed=5).randomSplit([0.8, 0.2])
train_df_notbanned, test_df_notbanned = nlpdf.filter(nlpdf.label==0).randomSplit([0.8, 0.2])
train_df_banned, test_df_banned = nlpdf.filter(nlpdf.label==1).randomSplit([0.8, 0.2])

In [ ]:
#unioning training sets and test sets
train_df = train_df_notbanned.union(train_df_banned)
test_df = test_df_notbanned.union(test_df_banned)

In [ ]:
# sample_train = train_df.cache()
# sample_test = test_df.cache()

# print("length of training set sample {}".format(sample_train.count()))
# print("length of test set {}".format(sample_test.count()))

# Visualizing Data <a id="2"></a>

## User Posts

In [ ]:
from pyspark.sql.functions import year, month, dayofmonth

stats = nlpdf.groupby(
    year('created_utc').alias('year'), 
    month('created_utc').alias('month')).count()

stats.show()

In [ ]:
stats_pdf = stats.toPandas()

In [ ]:
stats_pdf.head(5)

In [ ]:
pdf = stats_pdf.sort_values(by=['year', 'month'])

In [ ]:
pdf = pdf.reset_index(drop=True)
pdf.head()

In [ ]:
# convert into a str with leading 0
pdf['month'] = pdf['month'].apply(str)
pdf['month'] = pdf['month'].str.zfill(2)

pdf.head()

In [ ]:
pdf['year'] = pdf['year'].apply(str)
pdf['year_month'] = pdf['year'] + '-' + pdf['month']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ax = pdf.plot(kind='bar', y="count", x='year_month', align='edge', width=1.0, figsize=(20,10), color='blue', legend=False)
ax.grid()
plt.title("Posts made under topic \"Politics\"", size=24)
plt.ylabel("count", size=14)
plt.xlabel("year and month", size=14)

for p in ax.patches:
    if p.get_height() > 150000:
        ax.annotate(np.round(p.get_height(),decimals=2), (p.get_x()+p.get_width()/2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

## Word Cloud

In [ ]:
df_2016_11 = nlpdf.where(nlpdf.year == 2016).where(nlpdf.month == 11)
df_2019_2 = nlpdf.where(nlpdf.year == 2019).where(nlpdf.month == 2)
df_2019_9 = nlpdf.where(nlpdf.year == 2019).where(nlpdf.month == 9)

In [ ]:
from pyspark.sql.functions import array_contains, col, explode

# explode lists and retrieve all of the refined tokens
words_list_2016_11 = [x[0] for x in df_2016_11.select(explode('body_vec_token_nosw').alias('body_vec_token_nosw')).collect()]
words_list_2019_02 = [x[0] for x in df_2019_2.select(explode('body_vec_token_nosw').alias('body_vec_token_nosw')).collect()]
words_list_2019_09 = [x[0] for x in df_2019_9.select(explode('body_vec_token_nosw').alias('body_vec_token_nosw')).collect()]

In [ ]:
# count of total tags
len(words_list_2019_02)

In [ ]:
# get unique words
words_set = set(words_list_2019_02)
len(words_set)

In [ ]:
# use to get counts
import pandas as pd
pandas_series_2019_02 = pd.Series(words_list_2019_02)
pandas_series_2019_02.value_counts()

In [ ]:
# install wordcloud
#!conda install -c conda-forge wordcloud==1.4.1 --yes

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS

print ('Wordcloud is installed and imported!')

In [ ]:
#convert list to string and generate
comments_string_2016_11=(" ").join(words_list_2016_11)
comments_string_2019_02=(" ").join(words_list_2019_02)
comments_string_2019_09=(" ").join(words_list_2019_09)

In [ ]:
# instantiate a word cloud object
comments = WordCloud(
    background_color='white',
    max_words=2000)

# generate the word cloud
comments.generate(comments_string_2016_11)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# display the word cloud
plt.imshow(comments, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# generate the word cloud
comments.generate(comments_string_2019_02)

In [ ]:
# display the word cloud
plt.imshow(comments, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# generate the word cloud
comments.generate(comments_string_2019_09)

In [ ]:
# display the word cloud
plt.imshow(comments, interpolation='bilinear')
plt.axis('off')
plt.show()

## Posts within time interval bins

In [ ]:
df_interval_counts = nlpdf.where(nlpdf.year == 2019).groupby("year","month","interval").count()
df_interval_counts.sort(desc("interval")).show()

# Modeling <a id="3"></a>

In [38]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
start = time.time()
# Set parameters for Logistic Regression
lgr = LogisticRegression(maxIter=10, featuresCol = 'body_vec_tfidf', labelCol='label')

# Fit the model to the data.
lgrm = lgr.fit(train_df)
#lgrm = lgr.fit(sample_train)
end = time.time()
print('time to run: {}'.format(end - start))

time to run: 8.967144966125488


In [39]:
start = time.time()
# Given a dataset, predict each point's label, and show the results.
predictions = lgrm.transform(test_df)
#predictions = lgrm.transform(sample_test)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

print('prediction accuracy: {}'\
          .format(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))
print('prediction f1 score: {}'\
          .format(evaluator.evaluate(predictions, {evaluator.metricName: "f1"})))
end = time.time()
print('time to run: {}'.format(end - start))

prediction accuracy: 0.9626168224299065
prediction f1 score: 0.9626168224299065
time to run: 2.2132484912872314


In [40]:
# y_true = predictions.select('label').collect()
# y_pred = predictions.select('prediction').collect()
# true_list = [result.label for result in y_true]
# pred_list = [result.prediction for result in y_pred]

In [42]:
# from sklearn.metrics import classification_report, confusion_matrix
# print(classification_report(true_list, pred_list))

             precision    recall  f1-score   support

        0.0       0.98      0.98      0.98       105
        1.0       0.00      0.00      0.00         2

avg / total       0.96      0.96      0.96       107



In [43]:
from pyspark.ml.classification import RandomForestClassifier
# Set parameters for the Random Forest.
rfc = RandomForestClassifier(maxDepth=5, numTrees=15, impurity="gini", featuresCol='features',
                             labelCol="label")

# Fit the model to the data.
rfcm = rfc.fit(train_df)

# Given a dataset, predict each point's label, and show the results.
rfpredictions = rfcm.transform(test_df)

In [47]:
print('prediction accuracy: {}'\
          .format(evaluator.evaluate(rfpredictions, {evaluator.metricName: "accuracy"})))
print('prediction f1 score: {}'\
          .format(evaluator.evaluate(rfpredictions, {evaluator.metricName: "f1"})))

prediction accuracy: 1.0
prediction f1 score: 1.0


In [48]:
# y_true = rfpredictions.select('label').collect()
# y_pred = rfpredictions.select('prediction').collect()
# true_list = [result.label for result in y_true]
# pred_list = [result.prediction for result in y_pred]

In [49]:
# from sklearn.metrics import classification_report, confusion_matrix
# print(classification_report(true_list, pred_list))

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00       105
        1.0       1.00      1.00      1.00         2

avg / total       1.00      1.00      1.00       107



# Blah, blah, blah <a id="4"></a>